<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Monday 30 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 1: Staging data for analysis</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>2 </sup>Ioannis Bistinas, 
<sup>3, 4 </sup>Chantelle Burton, 
<sup>1 </sup>Tobias Marthews, 
<sup>5 </sup>Rhys Whitley
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Vrije Universiteit Amsterdam, Faculty of Earth and Life Sciences, Amsterdam, Netherlands
<br>
<sup>3 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>4 </sup>Geography, University of Exeter, Exeter, United Kingdom
<br>
<sup>5 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to process the separate netCDF4 files for the model drivers (X<sub>i=1, 2, ... M</sub>) and model target (Y) into a unified tabular data frame, exported as a compressed comma separated value (CSV) file. This file is subsequently used in the Bayesian inference study that forms the second notebook in this experiment. The advantage of the pre-processing the data separately to the analysis allows for it be quickly staged on demand. Of course other file formats may be more advantageous for greater compression (e.g. SQLite3 database file).
</p>
<br>
<b>You will need to run this notebook to prepare the dataest before you attempt the Bayesian analysis in Part 2</b>.
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
<hr>
</font>
</center>
</div>

## Load libraries

Changed

In [1]:
# data munging and analytical libraries 
import re
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset 

# graphical libraries
import matplotlib.pyplot as plt
%matplotlib inline

# set paths
inPath = "../data/n96_masked_data/outputs/"
outPath = "../data/globfire3.csv"

## Import and clean data

Set the directory path and look for all netcdf files that correspond to the model drivers and target.

In [2]:
driver_paths = [os.path.join(dp, f) for (dp, _, fn) in os.walk(inPath) for f in fn if f.endswith('.nc')]
driver_names = [re.search('^[a-zA-Z_]*', os.path.basename(fp)).group(0) for fp in driver_paths]

file_table = pd.DataFrame({'filepath': driver_paths, 'file_name': driver_names})
file_table

,filepath,file_name
0,../data/n96_masked_data/outputs/alpha_12monthM...,alpha_
1,../data/n96_masked_data/outputs/population_den...,population_density
2,../data/n96_masked_data/outputs/pasture2000-20...,pasture
3,../data/n96_masked_data/outputs/lightning_igni...,lightning_ignitions
4,../data/n96_masked_data/outputs/alpha2000-2014...,alpha
5,../data/n96_masked_data/outputs/emc2000-2014_m...,emc
6,../data/n96_masked_data/outputs/fire2000-2014_...,fire
7,../data/n96_masked_data/outputs/treecover2000-...,treecover
8,../data/n96_masked_data/outputs/cropland2000-2...,cropland
9,../data/n96_masked_data/outputs/vegcover2000-2...,vegcover


Define a function to extract the variable values from each netCDF4 file. Variables are flattened from a 3 dimensional array to 1 dimensional version, pooling all values both spatially and temporily. 

Don't know if this is the correct way to do this, but will come back to it once I understand the model (and its optimisation) better.

In [3]:
def nc_extract(fpath):
    print("Processing: {0}".format(fpath))
    with Dataset(fpath, 'r') as nc_file:
        gdata = np.array(nc_file.variables['variable'][:,:,:])
        gdata[gdata < -9E9] = np.nan
        gflat = gdata.flatten()
        if type(gdata) == np.ma.core.MaskedArray:
            return gflat[~gflat.mask].data
        else:
            return gflat.data

Execute the above function on all netCDF4 file paths.

In [4]:
values = [nc_extract(dp) for dp in driver_paths]

Processing: ../data/n96_masked_data/outputs/alpha_12monthMax2000-2014_masked.nc
Processing: ../data/n96_masked_data/outputs/population_density2000-2014_masked.nc
Processing: ../data/n96_masked_data/outputs/pasture2000-2014_masked.nc
Processing: ../data/n96_masked_data/outputs/lightning_ignitions2000-2014_masked.nc
Processing: ../data/n96_masked_data/outputs/alpha2000-2014_masked.nc
Processing: ../data/n96_masked_data/outputs/emc2000-2014_masked.nc
Processing: ../data/n96_masked_data/outputs/fire2000-2014_masked.nc
Processing: ../data/n96_masked_data/outputs/treecover2000-2014_masked.nc
Processing: ../data/n96_masked_data/outputs/cropland2000-2014_masked.nc
Processing: ../data/n96_masked_data/outputs/vegcover2000-2014_masked.nc


Turn this into a dataframe for the analysis.

In [5]:
# turn list into a dataframe
fire_df = pd.DataFrame(np.array(values).T, columns=driver_names)
fire_df.info()
# replace null flags with pandas null
#fire_df.replace(fire_df < -3e38, np.nan, inplace=True)
#fire_df[] = np.nan
# drop all null rows (are ocean and not needed in optim)
fire_df.dropna(inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4644864 entries, 0 to 4644863
Data columns (total 10 columns):
alpha_                 float32
population_density     float32
pasture                float32
lightning_ignitions    float32
alpha                  float32
emc                    float32
fire                   float32
treecover              float32
cropland               float32
vegcover               float32
dtypes: float32(10)
memory usage: 177.2 MB


Check that we've built it correctly.

In [10]:
fire_df.head()
# fire_df[fire_df['fire']>0.3]

,alpha_,population_density,pasture,lightning_ignitions,alpha,emc,fire,treecover,cropland,vegcover
12298,1.336481,2.225378,0.299290,1.709174,0.244917,0.000715,0.340622,0.152500,8.805742e-03,0.898750
12299,1.353586,0.836702,0.264825,2.106591,0.245200,0.000721,0.355084,0.165937,1.134923e-02,0.890625
12490,1.283013,1.886474,0.272335,1.992135,0.246668,0.038161,0.341019,0.209063,9.199035e-03,0.906875
39561,1.400698,16.397009,0.258890,1.583304,0.245783,0.005833,0.416503,0.235365,4.288723e-03,0.907396
64962,2.130781,0.001822,0.000000,0.470453,0.244954,0.025663,0.335115,0.002865,0.000000e+00,0.543958
65156,1.952929,0.008878,0.000000,0.296139,0.245082,0.005274,0.458055,0.021615,0.000000e+00,0.625365
65347,2.091664,0.000000,0.000000,0.427625,0.245460,0.010951,0.395937,0.011563,0.000000e+00,0.603125
65539,1.941899,0.000606,0.000000,0.642008,0.245591,0.009681,0.549921,0.002188,0.000000e+00,0.557396
65731,1.864927,0.000275,0.000000,0.709762,0.245737,0.007041,0.349803,0.000156,0.000000e+00,0.566302
65922,1.897231,0.000000,0.000000,0.796835,0.245953,0.011511,0.395899,0.000000,0.000000e+00,0.424115


Export this to disk to be used by the analysis notebook - used gzip compression to save on space. Beware, because of there are approximation 10 million rows of data, this may take some time.

In [7]:
savepath = os.path.expanduser(outPath)
fire_df.to_csv(savepath, index=False)

<div>
<br>
<br>
<br>
<center>
<font size="5">
<a style="font-weight: bold; size: 5" href="http://localhost:8888/notebooks/notebooks/bayesian_inference.ipynb">Part 2: click here</a>
</font>
</center>
</div>